In [ ]:

table=['CT_IND_AAAA_GN', 'CT_DEP_AAAA_GN', 'CT_IDE_AAAA_GN']
lib=["Table pathologies de la cartographie des pathologies pour l'année AAAA et l'algorithme N","Table dépenses de la cartographie des pathologies pour l'année AAAA et l'algorithme N","Table individus de la cartographie des pathologies pour l'année AAAA et l'algorithme N"]
tableexist={
    'tables':table,
    'tab_lib':lib,
    'produit':"CAROGRAPHIE_PATHOLOGIES"
}

In [ ]:
pip install tableschema

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 68 kB 3.2 MB/s 
     |████████████████████████████████| 72 kB 448 kB/s 
     |████████████████████████████████| 41 kB 496 kB/s 
     |████████████████████████████████| 132 kB 36.3 MB/s 
     |████████████████████████████████| 126 kB 56.9 MB/s 
     |████████████████████████████████| 79 kB 6.8 MB/s 
     |████████████████████████████████| 9.3 MB 64.8 MB/s 
     |████████████████████████████████| 127 kB 61.1 MB/s 
  Created wheel for linear-tsv: filename=linear_tsv-1.1.0-py3-none-any.whl size=7400 sha256=9f220358d8ef9332e5a05a3f59cc1de28abe841d56fbe7feeb4e1a265943f728
  Stored in directory: /root/.cache/pip/wheels/dd/0a/62/49ee84fab92057fb3e303fb1a59e75ccac6899b417f441305a
  Created wheel for unicodecsv: filename=unicodecsv-0.14.1-py3-none-any.whl size=10768 sha256=011700bdce04d7ec04ff14007e623924f8fc1256ce52c27409ad51a1b1070fe0
  Stored in directory:

In [ ]:
df

,variables,lib,Type,length,2012,2013,2014,2015,2016,2017,2018,2019,type,datecreated,datedeleted,datemissing,observation,rdg,table
0,AGE,Age du bénéficiaire,Num,8,X,X,X,X,X,X,X,X,integer,,,,,,CT_IND_AAAA_GN
1,ASS_AAH_TOP,Bénéficiaire de l'allocation adulte handicapé,Num,3,X,X,X,X,X,X,X,X,integer,,,,,,CT_IND_AAAA_GN
2,BEN_ACS_TOP,Bénéficiaire de l'aide à la complémentaire santé,Num,3,X,X,X,X,X,X,X,X,integer,,,,,,CT_IND_AAAA_GN
3,BEN_ALD_TOP,Bénéficiaire d'une exonération du ticket modér...,Num,3,X,X,X,X,X,X,X,X,integer,,,,,,CT_IND_AAAA_GN
4,BEN_DCD_AME,Année et mois de décès du bénéficiaire,Char,6,X,X,X,,,,,,string,,,,,,CT_IND_AAAA_GN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,TOP_PTRENFA_IND,Troubles psychiatriques ayant débuté dans l'en...,Num,3,X,X,X,X,X,X,X,X,integer,,,,,,CT_IND_AAAA_GN
181,TOP_RDIALYSE_IND,Dialyse chronique,Num,3,X,X,X,X,X,X,X,X,integer,,,,,,CT_IND_AAAA_GN
182,TOP_RTRANS_AIG,Transplantation rénale,Num,3,X,X,X,X,X,X,X,X,integer,,,,,,CT_IND_AAAA_GN
183,TOP_RTRANS_CHR,Suivi de transplantation rénale,Num,3,X,X,X,X,X,X,X,X,integer,,,,,,CT_IND_AAAA_GN


In [ ]:
annees=range(2012, 2020)

In [ ]:
annees

range(2012, 2020)

In [ ]:
from tableschema import Schema
import json
for table in tableexist['tables']:
  with open('{}.json'.format(table),'w', encoding='utf-8') as f:
    json.dump([],f, indent=4,ensure_ascii=False)
  import pandas as pd 
  df= pd.read_csv("{}.csv".format(table), sep=",",encoding='utf-8',na_values="")
  df.rename(columns={'Nom Variable':'variables','Libellé':'lib','Taille':'length'}, inplace=True)
  df=df.fillna('')


  ##### transformation des types 
  mapping_t = {
    "Char": "string",
    "Num":"integer",
    "Date":"date"
  }
  df["type"]=df["Type"].map(mapping_t)
  # Fonction qui permet de classer le type en number ou integer en fct de la taille 
  def numbint(x): 
    res = re.search(",", x)
    if res:
      return "number"
    return "integer"
  df["datecreated"]=''
  df["datedeleted"]=''
  df["datemissing"]=''
  df['observation']=''
  df['nomenclature']=''
  df["rdg"]=''
  df['table']=table
  
 # date of variable creation 
  annees=range(2012, 2020)
  for y in range(len(df['variables'])):
    for annee in annees:
      if df['{}'.format(annee)][y]=='X':
        df['datecreated'][y]=annee
        abs=range(annee,2020)
        for ab in abs:
          if df['{}'.format(ab)][y]=='':
            df['datedeleted'][y]=ab
            break
        break

 # Creation dy type oracle 
  mapping_to = {
    "string": "string",
    "date":"date",
    "integer":"number"
    
    }

  df["type_oracle"]=df["type"].map(mapping_to)

 
# Creation des fichiers json 
  for i in range(len(df['variables'])):
    if df['type'][i]=='number':
      df['type'][i]=df['length'][i].apply(numbint)    
     # les colonnes manquante  
  df=df.fillna('')
  
# create json file with the tables name

  for z in range(len(df['variables'])): 
    new_dict={
            "name": df["variables"][z],
            "description": df["lib"][z],
            "type": df["type"][z],
            "nomenclature": df["nomenclature"][z],
            "length": str(df["length"][z]),
            "format": "default",
            "dateCreated": str(df["datecreated"][z]),
            "dateDeleted": str(df["datedeleted"][z]),
            "dateMissing": df["datemissing"][z],
            "observation": df["observation"][z],
            "regle_gestion": df["rdg"][z],
            "type_oracle": df["type_oracle"][z]
       
        }
    
    with open('{}.json'.format(table),encoding='utf-8') as f:  
      data=json.load(f)
      data.append(new_dict)
    with open('{}.json'.format(table), 'w', encoding='utf-8') as f:
      json.dump(data, f, indent=4, ensure_ascii=False)
  # Mettons les fichiers sous format tableschema

from tableschema import Schema
for table in tableexist['tables']:
  with open('{}.json'.format(table), "r", encoding='utf-8') as f:
    descriptor = {'fields': json.load(f)}
    schema=Schema(descriptor) 
    schema.descriptor['name']=table
    schema.descriptor['title']=tableexist['tab_lib'][tableexist['tables'].index(table)]
    schema.descriptor['produit']=tableexist['produit']
    schema.descriptor['missingValues'] = ['']
    if table=='CT_IDE_AAAA_GN':
      schema.descriptor['primaryKey']= [
        "BEN_NIR_PSA",
        "BEN_RNG_GEM"
    ]
      schema.descriptor['foreignKeys'] = [
        {
            "fields": [
                "BEN_NIR_PSA",
                "BEN_RNG_GEM"
            ],
            "reference": {
                "resource": "IR_BEN_R",
                "fields": [
                    "BEN_NIR_PSA",
                    "BEN_RNG_GEM"
                ]
            },
            "description": "Bidirectionnel"
        }
    ]

    
                               
    schema.descriptor['foreignKeys'] =[
        {
            "fields": "id_carto",
            "reference": {
                "resource": "CT_IDE_AAAA_GN",
                "fields": "id_carto"
            }
        }
    ]

    schema.descriptor['history']={"dateCreated": str(schema.descriptor['fields'][0]['dateCreated']),
                                    "dateDeleted": "",
                                    "dateMissing": []}
    schema.descriptor['champ']=''
    schema.descriptor['observation']=''

    schema.commit()
    schema.save('{}.json'.format(table),ensure_ascii=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
